In [2]:
import pandas as pd
import requests
import json
import zipfile
import xmltodict
import time
from tqdm import tqdm
from io import BytesIO

## Data Load

### DART

In [2]:
# api_key 세팅
api_key = 'a47ee3f06af0391401258e8fc94b90296fa2d7f5'

In [4]:
# 1. 고유번호

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(url, params={'crtfc_key': api_key})
z = zipfile.ZipFile(BytesIO(res.content))

data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data_dart_1 = pd.DataFrame(data_dict.get('result', {}).get('list'))
data_dart_1

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
...,...,...,...,...
89361,01560457,삼원개발,None,20210506
89362,01556533,에이더블유파트너스,None,20210506
89363,01546299,동영와이케이,None,20210506
89364,00694605,미디어윌,None,20210506


In [3]:
#data_dart_1.to_pickle('data_dart_1.pkl')
data_dart_1 = pd.read_pickle('data_dart_1.pkl')
data_dart_1

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
...,...,...,...,...
89361,01560457,삼원개발,None,20210506
89362,01556533,에이더블유파트너스,None,20210506
89363,01546299,동영와이케이,None,20210506
89364,00694605,미디어윌,None,20210506


In [4]:
# stock_code 있는 건들만 선별
data_dart_1_nn = data_dart_1[data_dart_1.stock_code.notnull()]
data_dart_1_nn

,corp_code,corp_name,stock_code,modify_date
2013,00260985,한빛네트,036720,20170630
2025,00264529,엔플렉스,040130,20170630
2026,00358545,동서정보기술,055000,20170630
2788,00231567,애드모바일,032600,20170630
3896,00247939,씨모스,037600,20170630
...,...,...,...,...
89302,00521390,뉴파워프라즈마,144960,20211125
89311,00664853,제이씨케미칼,137950,20211126
89313,00307028,경남제약,053950,20211201
89344,00874195,코썬바이오,204990,20211027


In [26]:
# 2. 기업개황

data_dart_2 = pd.DataFrame()

for code in tqdm(data_dart_1_nn.corp_code.tolist()):
    url = f'https://opendart.fss.or.kr/api/company.json?crtfc_key={api_key}&corp_code={code}'
    res = requests.get(url)
    text = json.loads(res.text)
    
    if text['message'] == '정상':
        data_dart_2 = pd.concat([data_dart_2, pd.DataFrame.from_dict(text, orient='index').T], axis=0)
    
data_dart_2

100%|██████████████████████████████████████████████████████████████████████████████| 3401/3401 [04:58<00:00, 11.41it/s]


,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
0,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
0,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
0,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
0,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000,정상,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,,031-612-7600,031-612-7699,29271,19991007,12
0,000,정상,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,,02-6321-1333,027615404,2049,20060324,12
0,000,정상,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,,02-3490-5105,02)3490-5129,212,19981229,12
0,000,정상,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,,02-2628-0550,02-2628-0561,10797,20060808,12


In [5]:
#data_dart_2.to_pickle('data_dart_2.pkl')
data_dart_2 = pd.read_pickle('data_dart_2.pkl')
data_dart_2

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
0,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
0,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
0,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
0,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000,정상,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,,031-612-7600,031-612-7699,29271,19991007,12
0,000,정상,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,,02-6321-1333,027615404,2049,20060324,12
0,000,정상,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,,02-3490-5105,02)3490-5129,212,19981229,12
0,000,정상,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,,02-2628-0550,02-2628-0561,10797,20060808,12


In [12]:
# 3. 직원현황

data_dart_3 = pd.DataFrame()

for code in tqdm(data_dart_1_nn.corp_code.tolist()):
    for year in ['2020', '2019']:
        url = f'https://opendart.fss.or.kr/api/empSttus.json?crtfc_key={api_key}&corp_code={code}&bsns_year={year}&reprt_code=11011'
        res = requests.get(url)
        text = json.loads(res.text)
        
        if text['message'] == '정상':
            data_dart_3 = pd.concat([data_dart_3, pd.DataFrame(text['list'])], axis=0)
        else:
            break
            
        time.sleep(0.5)
    
    time.sleep(1)

data_dart_3

100%|████████████████████████████████████████████████████████████████████████████| 3401/3401 [1:46:57<00:00,  1.89s/it]


,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20210316000210,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,10,-,-,-,10,8.64,"943,156","94,316"
1,20210316000210,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,4,-,-,-,4,7.65,"319,674","79,919"
2,20210316000210,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,6.07,"521,380","130,345"
3,20210316000210,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,1,-,-,-,1,0.25,"13,514","13,514"
4,20210316000210,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,4.92,"853,502","121,929"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,-,여,제작,-,-,-,68,-,-,-,68,2.3,"1,953,000,000","32,000,000"
2,20210331003230,K,01264438,자이언트스텝,-,남,연구개발,-,-,-,14,-,-,-,14,1.8,"712,000,000","56,000,000"
3,20210331003230,K,01264438,자이언트스텝,-,여,연구개발,-,-,-,6,-,-,-,6,1.1,"146,000,000","30,000,000"
4,20210331003230,K,01264438,자이언트스텝,-,남,관리,-,-,-,11,2,-,-,13,4.1,"451,000,000","47,000,000"


In [5]:
#data_dart_3.to_pickle('data_dart_3.pkl')
data_dart_3 = pd.read_pickle('data_dart_4.pkl')
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20210316000210,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,10,-,-,-,10,8.64,"943,156","94,316"
1,20210316000210,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,4,-,-,-,4,7.65,"319,674","79,919"
2,20210316000210,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,6.07,"521,380","130,345"
3,20210316000210,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,1,-,-,-,1,0.25,"13,514","13,514"
4,20210316000210,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,4.92,"853,502","121,929"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,-,여,제작,-,-,-,68,-,-,-,68,2.3,"1,953,000,000","32,000,000"
2,20210331003230,K,01264438,자이언트스텝,-,남,연구개발,-,-,-,14,-,-,-,14,1.8,"712,000,000","56,000,000"
3,20210331003230,K,01264438,자이언트스텝,-,여,연구개발,-,-,-,6,-,-,-,6,1.1,"146,000,000","30,000,000"
4,20210331003230,K,01264438,자이언트스텝,-,남,관리,-,-,-,11,2,-,-,13,4.1,"451,000,000","47,000,000"


In [4]:
#data_dart_3.to_pickle('data_dart_3.pkl')
data_dart_3 = pd.read_pickle('data_dart_4.pkl')
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20210316000210,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,10,-,-,-,10,8.64,"943,156","94,316"
1,20210316000210,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,4,-,-,-,4,7.65,"319,674","79,919"
2,20210316000210,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,6.07,"521,380","130,345"
3,20210316000210,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,1,-,-,-,1,0.25,"13,514","13,514"
4,20210316000210,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,4.92,"853,502","121,929"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,-,여,제작,-,-,-,68,-,-,-,68,2.3,"1,953,000,000","32,000,000"
2,20210331003230,K,01264438,자이언트스텝,-,남,연구개발,-,-,-,14,-,-,-,14,1.8,"712,000,000","56,000,000"
3,20210331003230,K,01264438,자이언트스텝,-,여,연구개발,-,-,-,6,-,-,-,6,1.1,"146,000,000","30,000,000"
4,20210331003230,K,01264438,자이언트스텝,-,남,관리,-,-,-,11,2,-,-,13,4.1,"451,000,000","47,000,000"


In [6]:
data_dart_3.corp_code.unique().size

2544

In [7]:
data_dart_3.corp_name.unique().tolist()

['엑세스바이오',
 '지오엠씨',
 '클루넷',
 '글로벌에스엠',
 '씨케이에이치',
 'GRT',
 '윙입푸드',
 '루트원플러스',
 '로스웰',
 '골든센츄리',
 '에스티씨라이프',
 '동양텔레콤',
 '에임하이글로벌',
 '디비엘',
 '더스텔라',
 '코오롱티슈진',
 'SBI핀테크솔루션즈',
 '자유투어',
 '영신금속',
 '웹젠',
 '우리넷',
 '마이크로디지탈',
 '손오공',
 '새로닉스',
 '교보9호스팩',
 '네추럴FNP',
 '로보로보',
 '교보10호기업인수목적',
 '아이비김영',
 '메디안디노스틱',
 '씨앤에스링크',
 '유나이티드',
 '프럼파스트',
 'HDC현대산업개발',
 '지니언스',
 '시노펙스',
 '모나리자',
 '티웨이홀딩스',
 '이수화학',
 '씨앤지하이테크',
 '삼화왕관',
 '이지스밸류플러스리츠',
 '피엔에이치테크',
 '비엔디생활건강',
 '하림지주',
 '아모텍',
 '삼영이엔씨',
 '코리아센터',
 '대상홀딩스',
 '애니젠',
 '엠투아이',
 '흥국화재',
 '이트론',
 '바이오리더스',
 '넥스지',
 '케이티스카이라이프',
 '아바코',
 '미원화학',
 '메타랩스',
 '에스알바이오텍',
 '티와이홀딩스',
 '나눔테크',
 '팜스빌',
 '동일고무벨트',
 '성도이엔지',
 '한국맥널티',
 '바다로19호',
 '폴루스바이오팜',
 'SK6호스팩',
 'BNK금융지주',
 '위드텍',
 '다원넥스뷰',
 '심텍홀딩스',
 '솔트룩스',
 '팬엔터테인먼트',
 '솔루엠',
 '셀트리온헬스케어',
 '라온피플',
 '파미셀',
 '아이앤씨',
 '코미코',
 '네오크레마',
 '모트렉스',
 '에스티오',
 '텔레칩스',
 'HRS',
 '덕성',
 'KG케미칼',
 '엔지스테크널러지',
 '콤텍시스템',
 '파워풀엑스',
 '한국철강',
 'KISCO홀딩스',
 '핑거',
 '이엠티',
 '메디아나',
 '와이엠씨',
 '예선테크',
 '티에스아이',
 '마이크로프랜드',
 '